In [1]:
#importing all the required libraries and modules
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pandas.tseries.holiday import USFederalHolidayCalendar
import scipy.stats
import matplotlib.dates as mdates
from google.cloud import bigquery

In [2]:
#connecting notebook to gcloud bigquery to query data.
cred_file = r"C:\Users\V0V01S7\AppData\Roaming\gcloud\legacy_credentials\venkatasaiyashaswi.v@walmart.com\adc.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=cred_file
def query_bq(QUERY):
    client = bigquery.Client(project='wmt-edw-sandbox')
    query_job = client.query(QUERY) # API request
    rows = query_job.result() # Waits for query to finish
    df = rows.to_dataframe()
    return df

In [40]:
code2 = """
SELECT A.DRVR_USER_ID, A.MARKET_NM,
C.DRVR_BIRTH_DT,
from `wmt-edw-prod.WW_GEC_VM.DRVR` A inner join  `wmt-edw-prod.WW_GEC_SECURE.DRVR` C ON A.DRVR_USER_ID = C.DRVR_USER_ID
WHERE A.MARKET_NM in ('Walmart San Jose North','Walmart San Jose South','Walmart Orlando','Walmart Virginia Beach', 'Walmart Tulsa','Walmart South Minneapolis') AND B.ACTV_NM = "ONLINE"
"""
df2 = query_bq(code2)
df2.head()

KeyboardInterrupt: 

In [5]:
df = pd.read_csv('Downloads/weekday-age.csv')

In [7]:
df = df[df["DRVR_USER_ID"].str.contains("walmart|faisal") == False]
df = df[df["ACTV_DUR_MIN_QTY"] > 0 ]
df = df.drop_duplicates()

In [8]:
df

,DRVR_USER_ID,MARKET_NM,DRVR_BIRTH_DT,ACTV_DUR_MIN_QTY,ACTV_DT
0,donnitap@msn.com,Walmart Virginia Beach,1973-05-24,0.13,2021-03-05
2,m.leanne08@gmail.com,Walmart Virginia Beach,1991-01-10,70.35,2021-03-05
199,cmoore0830@yahoo.com,Walmart Virginia Beach,1982-08-29,173.15,2021-03-05
200,m.leanne08@gmail.com,Walmart Virginia Beach,1991-01-10,86.68,2021-03-05
201,william.miles.wm@gmail.com,Walmart Virginia Beach,1977-10-18,2.20,2021-03-05
...,...,...,...,...,...
69800,tedsmith147@gmail.com,Walmart Virginia Beach,1953-09-05,34.45,2021-05-25
69801,libiasanchezmujica58@gmail.com,Walmart Orlando,1958-12-27,124.02,2021-05-25
69802,ceivings.libny@gmail.com,Walmart Virginia Beach,1971-11-19,24.92,2021-05-25
69972,ceivings.libny@gmail.com,Walmart Virginia Beach,1971-11-19,0.15,2021-05-25


In [12]:
def age(birthdate):
    today = datetime.today()
    age = today.year - birthdate.year
    return age

In [15]:
df['DRVR_BIRTH_DT'] = pd.to_datetime(df['DRVR_BIRTH_DT'])
df.dtypes

DRVR_USER_ID                object
MARKET_NM                   object
DRVR_BIRTH_DT       datetime64[ns]
ACTV_DUR_MIN_QTY           float64
ACTV_DT                     object
dtype: object

In [16]:
df['age'] = df['DRVR_BIRTH_DT'].apply(lambda x: age(x))

,DRVR_USER_ID,MARKET_NM,DRVR_BIRTH_DT,ACTV_DUR_MIN_QTY,ACTV_DT,age
0,donnitap@msn.com,Walmart Virginia Beach,1973-05-24,0.13,2021-03-05,49.0
2,m.leanne08@gmail.com,Walmart Virginia Beach,1991-01-10,70.35,2021-03-05,31.0
199,cmoore0830@yahoo.com,Walmart Virginia Beach,1982-08-29,173.15,2021-03-05,40.0
200,m.leanne08@gmail.com,Walmart Virginia Beach,1991-01-10,86.68,2021-03-05,31.0
201,william.miles.wm@gmail.com,Walmart Virginia Beach,1977-10-18,2.20,2021-03-05,45.0
...,...,...,...,...,...,...
69800,tedsmith147@gmail.com,Walmart Virginia Beach,1953-09-05,34.45,2021-05-25,69.0
69801,libiasanchezmujica58@gmail.com,Walmart Orlando,1958-12-27,124.02,2021-05-25,64.0
69802,ceivings.libny@gmail.com,Walmart Virginia Beach,1971-11-19,24.92,2021-05-25,51.0
69972,ceivings.libny@gmail.com,Walmart Virginia Beach,1971-11-19,0.15,2021-05-25,51.0


In [21]:
df = df.groupby(['MARKET_NM','DRVR_USER_ID','age','ACTV_DT'],as_index=False).sum()

In [23]:
df.dtypes

MARKET_NM            object
DRVR_USER_ID         object
age                 float64
ACTV_DT              object
ACTV_DUR_MIN_QTY    float64
dtype: object

In [24]:
df.head(10)

,MARKET_NM,DRVR_USER_ID,age,ACTV_DT,ACTV_DUR_MIN_QTY
0,Walmart Orlando,1219paola@gmail.com,30.0,2021-03-14,192.87
1,Walmart Orlando,1219paola@gmail.com,30.0,2021-05-03,51.68
2,Walmart Orlando,1219paola@gmail.com,30.0,2021-06-18,246.75
3,Walmart Orlando,69blesss@gmail.com,63.0,2021-07-02,530.43
4,Walmart Orlando,69blesss@gmail.com,63.0,2021-07-03,247.95
5,Walmart Orlando,69blesss@gmail.com,63.0,2021-07-12,373.45
6,Walmart Orlando,a.gordon14@yahoo.com,28.0,2021-04-12,22.15
7,Walmart Orlando,a.gordon14@yahoo.com,28.0,2021-04-14,44.90
8,Walmart Orlando,a.gordon14@yahoo.com,28.0,2021-05-25,1.02
9,Walmart Orlando,aaliyahmarks06@gmail.com,28.0,2021-05-25,242.00


In [25]:
df['ACTV_DT'] = pd.to_datetime(df['ACTV_DT'])
df['dow'] = df['ACTV_DT'].dt.dayofweek
df.head()

,MARKET_NM,DRVR_USER_ID,age,ACTV_DT,ACTV_DUR_MIN_QTY,dow
0,Walmart Orlando,1219paola@gmail.com,30.0,2021-03-14,192.87,6
1,Walmart Orlando,1219paola@gmail.com,30.0,2021-05-03,51.68,0
2,Walmart Orlando,1219paola@gmail.com,30.0,2021-06-18,246.75,4
3,Walmart Orlando,69blesss@gmail.com,63.0,2021-07-02,530.43,4
4,Walmart Orlando,69blesss@gmail.com,63.0,2021-07-03,247.95,5


In [38]:
max(df['date'])

Timestamp('2021-07-19 00:00:00')

In [26]:
df = df.rename(columns={'MARKET_NM':'market','DRVR_USER_ID':'driver','ACTV_DT':'date','ACTV_DUR_MIN_QTY':'minutes'})

In [23]:
df.to_csv('Desktop/driver_actv.csv')

In [27]:
time_line = pd.date_range(df["date"].min(),df["date"].max(),freq='D')
time_line = time_line.to_series()
ts = pd.DataFrame(time_line,columns = ["date"])

In [28]:
timeline = pd.merge(ts,df,on='date',how='left')

In [32]:
timeline

,date,market,driver,age,minutes,dow
0,2021-03-03,Walmart Orlando,noelia121726@gmail.com,31.0,201.97,2.0
1,2021-03-03,Walmart Orlando,veroestetic_59@hotmail.com,63.0,0.27,2.0
2,2021-03-03,Walmart Virginia Beach,ajspeller@gmail.com,48.0,426.70,2.0
3,2021-03-03,Walmart Virginia Beach,allshowbiz@gmail.com,44.0,353.90,2.0
4,2021-03-03,Walmart Virginia Beach,bmperkins155@gmail.com,31.0,428.49,2.0
...,...,...,...,...,...,...
9340,2021-07-19,Walmart Virginia Beach,vnssaflowers@aol.com,28.0,34.80,0.0
9341,2021-07-19,Walmart Virginia Beach,whiteadw26@gmail.com,49.0,298.40,0.0
9342,2021-07-19,Walmart Virginia Beach,yancarloschiu@gmail.com,39.0,246.23,0.0
9343,2021-07-19,Walmart Virginia Beach,yavuzrasul@gmail.com,42.0,511.08,0.0


In [37]:
timeline.tail()

,date,market,driver,age,minutes,dow
9340,2021-07-19,Walmart Virginia Beach,vnssaflowers@aol.com,28.0,34.80,0.0
9341,2021-07-19,Walmart Virginia Beach,whiteadw26@gmail.com,49.0,298.40,0.0
9342,2021-07-19,Walmart Virginia Beach,yancarloschiu@gmail.com,39.0,246.23,0.0
9343,2021-07-19,Walmart Virginia Beach,yavuzrasul@gmail.com,42.0,511.08,0.0
9344,2021-07-19,Walmart Virginia Beach,ymcanady@gmail.com,38.0,151.33,0.0


In [37]:
timeline['minutes'] = timeline['minutes'].fillna(0)

In [38]:
timeline

,date,market,driver,minutes,dow
0,2021-02-13,Walmart Virginia Beach,manifest219@gmail.com,196.630000000,5.0
1,2021-02-14,Walmart Virginia Beach,manifest219@gmail.com,247.880000000,6.0
2,2021-02-15,NaN,NaN,0,NaN
3,2021-02-16,NaN,NaN,0,NaN
4,2021-02-17,NaN,NaN,0,NaN
...,...,...,...,...,...
291134,2022-07-27,Walmart Virginia Beach,yavuzrasul@gmail.com,96.460000000,2.0
291135,2022-07-27,Walmart Virginia Beach,yelhsa.n.s@gmail.com,504.490000000,2.0
291136,2022-07-27,Walmart Virginia Beach,yodiancurry@gmail.com,452.820000000,2.0
291137,2022-07-27,Walmart Virginia Beach,zoeeestaxx@icloud.com,310.140000000,2.0


In [39]:
timeline['dow'] = timeline['date'].dt.dayofweek

In [40]:
timeline

,date,market,driver,minutes,dow
0,2021-02-13,Walmart Virginia Beach,manifest219@gmail.com,196.630000000,5
1,2021-02-14,Walmart Virginia Beach,manifest219@gmail.com,247.880000000,6
2,2021-02-15,NaN,NaN,0,0
3,2021-02-16,NaN,NaN,0,1
4,2021-02-17,NaN,NaN,0,2
...,...,...,...,...,...
291134,2022-07-27,Walmart Virginia Beach,yavuzrasul@gmail.com,96.460000000,2
291135,2022-07-27,Walmart Virginia Beach,yelhsa.n.s@gmail.com,504.490000000,2
291136,2022-07-27,Walmart Virginia Beach,yodiancurry@gmail.com,452.820000000,2
291137,2022-07-27,Walmart Virginia Beach,zoeeestaxx@icloud.com,310.140000000,2


In [41]:
timeline.to_csv('Desktop/driver_actv.csv')

In [46]:
mjj = timeline[timeline['date'] >= '2022-05-01']

In [110]:
mjj

,date,market,driver,minutes,dow
208923,2022-05-01,Walmart Orlando,aaronwork407@gmail.com,304.170000000,6
208924,2022-05-01,Walmart Orlando,acordidos@gmail.com,339.260000000,6
208925,2022-05-01,Walmart Orlando,adrianafariasrocha@icloud.com,466.400000000,6
208926,2022-05-01,Walmart Orlando,agmirete68@yahoo.es,173.050000000,6
208927,2022-05-01,Walmart Orlando,aidinbarreno@hotmail.com,738.960000000,6
...,...,...,...,...,...
291134,2022-07-27,Walmart Virginia Beach,yavuzrasul@gmail.com,96.460000000,2
291135,2022-07-27,Walmart Virginia Beach,yelhsa.n.s@gmail.com,504.490000000,2
291136,2022-07-27,Walmart Virginia Beach,yodiancurry@gmail.com,452.820000000,2
291137,2022-07-27,Walmart Virginia Beach,zoeeestaxx@icloud.com,310.140000000,2


In [50]:
dfs = {}
for ID in mjj['driver'].value_counts().index.tolist():
    dfs[ID] = mjj.loc[mjj['driver'] == ID]

In [52]:
len(dfs)

3306

In [60]:
dfs['yavuzrasul@gmail.com']

,date,market,driver,minutes,dow
209636,2022-05-01,Walmart Virginia Beach,yavuzrasul@gmail.com,30.880000000,6
210428,2022-05-02,Walmart Virginia Beach,yavuzrasul@gmail.com,178.300000000,0
211274,2022-05-03,Walmart Virginia Beach,yavuzrasul@gmail.com,117.450000000,1
212125,2022-05-04,Walmart Virginia Beach,yavuzrasul@gmail.com,41.590000000,2
212974,2022-05-05,Walmart Virginia Beach,yavuzrasul@gmail.com,40.900000000,3
...,...,...,...,...,...
287170,2022-07-23,Walmart Virginia Beach,yavuzrasul@gmail.com,11.250000000,5
288097,2022-07-24,Walmart Virginia Beach,yavuzrasul@gmail.com,18.270000000,6
289104,2022-07-25,Walmart Virginia Beach,yavuzrasul@gmail.com,131.580000000,0
290104,2022-07-26,Walmart Virginia Beach,yavuzrasul@gmail.com,193.810000000,1


In [61]:
def final_ts(df):
    time_line = pd.date_range(df["date"].min(),df["date"].max(),freq='D')
    time_line = time_line.to_series()
    ts = pd.DataFrame(time_line,columns = ["date"])
    ts = ts.set_index("date")
    ts = pd.merge(ts,df,how="left",on=["date"])
    ts["minutes"] = ts["minutes"].fillna(0)
    ts["driver"] = ts["driver"].iloc[0]
    ts["market"] = ts["market"].iloc[0]
    ts['dow'] = ts['date'].dt.dayofweek
    ts = ts.drop_duplicates()
    return ts

In [64]:
dfs_set = {}
for i in dfs.keys():
    dfs_set[i] = pd.DataFrame(final_ts(dfs[i]))

In [66]:
dff = pd.concat([dfs_set[i] for i in dfs_set.keys()], ignore_index=False)

In [68]:
dff.to_csv('Desktop/driver_actv.csv')

In [74]:
temp = dff[dff['driver'] == 'varsi.littlejohn@gmail.com']

In [80]:
temp.groupby('dow').size()

dow
0    13
1    13
2    13
3    12
4    12
5    12
6    13
dtype: int64

In [81]:
temp[temp['minutes']>0].groupby('dow').size().divide(temp.groupby('dow').size())

dow
0    0.692308
1    0.846154
2    0.923077
3    1.000000
4    1.000000
5    0.750000
6    0.692308
dtype: float64

In [82]:
dayprobs = {}
for i in dfs_set.keys():
    temp = dfs_set[i]
    dayprobs[i] = temp[temp['minutes']>0].groupby('dow').size().divide(temp.groupby('dow').size())

In [88]:
pdf = pd.DataFrame.from_dict(dayprobs)

In [90]:
pdf = pdf.fillna(0)

In [98]:
pdf = pdf.transpose()

In [100]:
pdf = pdf.reset_index(drop = False)

In [107]:
pdf = pdf.rename(columns = {'index': 'driver','dow':'index'})

In [109]:
pdf.to_csv('Desktop/dayprobs.csv')

In [1]:
pdf

NameError: name 'pdf' is not defined